In [1]:
import pandas as pd
import random
import math
import osmnx as ox
import networkx as nx
import folium

In [5]:
# CONFIGURATION
# ------------------------------------------
# Central restaurant location (Berlin city center)
CENTRAL_LAT, CENTRAL_LON = 52.5200, 13.4050

# Number of customer locations to generate (excluding the restaurant)
NUM_CUSTOMERS = 500

# Radius (in meters) around the central point within which to sample customers
RADIUS_METERS = 5000  

# Earth's radius in meters (for converting meters to degrees)
EARTH_RADIUS = 6371000  

# Average cycling speed (km/h)
BIKE_SPEED_KPH = 30.0

# Filenames for outputs
CUSTOMER_CSV = "new_customer_locations.csv"
TRAVEL_TIME_CSV = "travel_time_matrix.csv"
MAP_HTML      = "customer_locations_map.html"


In [7]:
# FUNCTION: Generate one random point within a circle
# ------------------------------------------
def generate_random_point(lat_center: float, lon_center: float, radius: float):
    """
    Generates a random latitude/longitude pair within "radius" meters of (lat_center, lon_center).

    Uses a uniform distribution over the circle area.
    """
    # Random radius (with sqrt to ensure uniform area distribution)
    r = radius * math.sqrt(random.random())
    # Random angle [0, 2π)
    theta = random.random() * 2 * math.pi

    # Offset in radians
    delta_lat = (r * math.cos(theta)) / EARTH_RADIUS
    delta_lon = (r * math.sin(theta)) / (EARTH_RADIUS * math.cos(math.radians(lat_center)))

    # Convert offsets from radians to degrees
    lat = lat_center + math.degrees(delta_lat)
    lon = lon_center + math.degrees(delta_lon)
    return lat, lon

In [13]:
# ------------------------------------------
# STEP 1: Generate Customer + Restaurant Locations (lat/lon only)
# ------------------------------------------
locations = []

# 1a) Restaurant (id = 0) at the exact center
locations.append({
    "id":     0,
    "lat":    CENTRAL_LAT,
    "lon":    CENTRAL_LON
})

# 1b) Generate NUM_CUSTOMERS random points within RADIUS_METERS
for i in range(1, NUM_CUSTOMERS + 1):
    lat, lon = generate_random_point(CENTRAL_LAT, CENTRAL_LON, RADIUS_METERS)
    locations.append({
        "id":    i,
        "lat":   lat,
        "lon":   lon
    })

# Build a DataFrame without osmid yet
df_locations = pd.DataFrame(locations, columns=["id", "lat", "lon"])

In [15]:
# Print first 5 locations to verify
print("First 5 generated locations (including restaurant at index 0):")
print(df_locations.head())

First 5 generated locations (including restaurant at index 0):
   id        lat        lon
0   0  52.520000  13.405000
1   1  52.510708  13.348160
2   2  52.550885  13.440514
3   3  52.520859  13.371159
4   4  52.498688  13.429820


In [19]:
# STEP 2: Download Berlin Street Network via OSMnx
# ------------------------------------------
print("Downloading Berlin cycling network (within 5 km)…")
# Use driving network as base
G = ox.graph_from_point(
    (CENTRAL_LAT, CENTRAL_LON),
    dist=RADIUS_METERS,
    network_type="drive"
)

In [21]:
# STEP 3: Assign Uniform Bike Speed and Compute Travel Times
# ------------------------------------------
print("Assigning bike travel times to each edge…")
bike_speed_ms = (BIKE_SPEED_KPH * 1000) / 3600.0

for u, v, key, data in G.edges(keys=True, data=True):
    length_m = data.get('length', 0)
    travel_time_sec = length_m / bike_speed_ms
    data['travel_time'] = travel_time_sec

Assigning bike travel times to each edge…


In [23]:
# STEP 4: Snap Each Location to Its Nearest OSM Node and Set osmid
# ------------------------------------------
print("Finding nearest network node for each location and setting osmid…")
node_ids = []
for idx, row in df_locations.iterrows():
    node_id = ox.distance.nearest_nodes(G, X=row["lon"], Y=row["lat"])
    node_ids.append(node_id)
# Add osmid column based on nearest node
df_locations['osmid'] = node_ids

Finding nearest network node for each location and setting osmid…


In [25]:
# STEP 5: Compute Full-Pairwise Travel Time Matrix (minutes)
# ------------------------------------------
print("Computing travel time matrix (in minutes)…")
# Prepare a DataFrame indexed/columned by id
travel_time_df = pd.DataFrame(
    index=df_locations['id'],
    columns=df_locations['id'],
    dtype=float
)

# For each source node, compute shortest paths
for i, src_node in enumerate(node_ids):
    src_id = df_locations.at[i, 'id']
    lengths_sec = nx.single_source_dijkstra_path_length(G, src_node, weight='travel_time')
    for j, tgt_node in enumerate(node_ids):
        tgt_id = df_locations.at[j, 'id']
        t_sec = lengths_sec.get(tgt_node, float('nan'))
        travel_time_df.at[src_id, tgt_id] = t_sec / 60.0

# Save travel time matrix to CSV
travel_time_df.to_csv(TRAVEL_TIME_CSV, float_format="%.2f")
print(f"Saved travel time matrix to '{TRAVEL_TIME_CSV}'")

Computing travel time matrix (in minutes)…
Saved travel time matrix to 'travel_time_matrix.csv'


In [27]:
# STEP 6: Save Customer Locations with osmid to CSV
# ------------------------------------------
# Reorder columns to id, osmid, lat, lon
df_locations = df_locations[['id', 'osmid', 'lat', 'lon']]
df_locations.to_csv(CUSTOMER_CSV, index=False)
print(f"Saved {len(df_locations)} locations with osmid to '{CUSTOMER_CSV}'")


Saved 501 locations with osmid to 'new_customer_locations.csv'


In [29]:
# STEP 7: Plot All Points on a Folium Map
# ------------------------------------------
print("Creating interactive map (HTML)…")
m = folium.Map(location=[CENTRAL_LAT, CENTRAL_LON], zoom_start=13)

# 7a) Restaurant marker (id = 0) in red
folium.CircleMarker(
    location=[CENTRAL_LAT, CENTRAL_LON],
    radius=7,
    color="red",
    fill=True,
    fill_color="red",
    popup=f"Restaurant (ID 0, OSMID {df_locations.at[0, 'osmid']})"
).add_to(m)

# 7b) Customer markers in blue
for idx, row in df_locations[df_locations['id'] != 0].iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        color="blue",
        fill=True,
        popup=f"Customer ID {int(row['id'])}, OSMID {int(row['osmid'])}"
    ).add_to(m)

# Save to HTML
m.save(MAP_HTML)
print(f"Saved map to '{MAP_HTML}'")

print("All done! Files generated:")
print(f" • Customer locations CSV: {CUSTOMER_CSV}")
print(f" • Travel time matrix CSV: {TRAVEL_TIME_CSV}")
print(f" • Interactive map HTML:    {MAP_HTML}")

Creating interactive map (HTML)…
Saved map to 'customer_locations_map.html'
All done! Files generated:
 • Customer locations CSV: new_customer_locations.csv
 • Travel time matrix CSV: travel_time_matrix.csv
 • Interactive map HTML:    customer_locations_map.html


dwqdas